In [13]:
import pulp
import networkx as nx
import itertools
import time

In [14]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [15]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1])  
            G.add_edge(node1, node2) 
    return G

In [ ]:
def pulp1(G):
   
    n = len(G.nodes)
    model = pulp.LpProblem("Metric_Dimension_Minimize_Set_Size", pulp.LpMinimize)

    y = pulp.LpVariable.dicts("y", G.nodes, cat="Binary")

    # Ciljna funkcija
    model += pulp.lpSum(y[j] for j in G.nodes), "Objective"

    # Ograničenja: Svaki par čvorova mora biti razrješen
    for u, v in itertools.combinations(G.nodes, 2):
        model += pulp.lpSum(abs(nx.shortest_path_length(G, u, j) - nx.shortest_path_length(G, v, j)) * y[j]
                             for j in G.nodes) >= 1, f"Resolve_{u}_{v}"
        
    model.solve(pulp.PULP_CBC_CMD(msg=False))

    # Prikaz rezultata
    resolving_set = [j for j in G.nodes if pulp.value(y[j]) == 1]
    return resolving_set

In [ ]:
def pulp2(G):
 
    n = len(G.nodes)

    # Početna gornja granica
    s = n
    best_resolving_set = None

    while s > 1:  
        # Kreiranje ILP modela
        model = pulp.LpProblem("Metric_Dimension_Minimize_Unresolved_Pairs", pulp.LpMinimize)

        # Binarne promenljive y[j] = 1 ako je čvor j u rezolventnom skupu
        y = pulp.LpVariable.dicts("y", G.nodes, cat="Binary")

        # Binarne promenljive z[u, v] = 1 ako par (u, v) nije razrešen
        z = pulp.LpVariable.dicts("z", itertools.combinations(G.nodes, 2), cat="Binary")

        # Ciljna funkcija
        model += pulp.lpSum(z[u, v] for u, v in itertools.combinations(G.nodes, 2)), "Min_Unresolved_Pairs"

        # Ograničenje: kardinalnost skupa mora biti tačno s
        model += pulp.lpSum(y[j] for j in G.nodes) == s, "Fixed_Cardinality"

        # Ograničenja za razrješavanje parova
        for u, v in itertools.combinations(G.nodes, 2):
            model += pulp.lpSum(abs(nx.shortest_path_length(G, u, j) - nx.shortest_path_length(G, v, j)) * y[j]
                                 for j in G.nodes) + z[u, v] >= 1, f"Resolve_{u}_{v}"

        
        model.solve(pulp.PULP_CBC_CMD(msg=False))

        # Ako rješenje nije pronađeno, prekidamo petlju
        if pulp.LpStatus[model.status] != "Optimal":
            break

        # Uzimamo trenutno rješenje
        resolving_set = [j for j in G.nodes if pulp.value(y[j]) == 1]

        # Provjera da li su svi parovi razrješeni
        unresolved_pairs = sum(pulp.value(z[u, v]) for u, v in itertools.combinations(G.nodes, 2))

        if unresolved_pairs == 0:
            # Ako nema nerješenih parova, čuvamo rješenje i pokušavamo da smanjimo s
            best_resolving_set = resolving_set
            s -= 1
        else:
            # Ako postoje nerješeni parovi, vraćamo najbolje do sada rješenje
            return best_resolving_set

    return best_resolving_set if best_resolving_set else []


In [ ]:
G = nx.Graph()
G = load_graph('grafovi\csp\csp100.txt')
if not nx.is_connected(G):
    print("Graf nije povezan!")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()

start_time1 = time.time()
result1 = pulp1(G)
end_time1 = time.time()
print("Optimalni rješavajući skup (Prvi model):", result1, len(result1))
print("Vrijeme izvršavanja (Prvi model):", end_time1 - start_time1, "sekundi")


start_time2 = time.time()
result2 = pulp2(G)
end_time2 = time.time()
print("Optimalni rješavajući skup (Drugi model):", result2, len(result2))
print("Vrijeme izvršavanja (Drugi model):", end_time2 - start_time2, "sekundi")

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_387984\1791757485.py:2: SyntaxWarning: invalid escape sequence '\c'
  G = load_graph('grafovi\csp\csp100.txt')


Graf nije povezan!
Optimalni rešavajući skup (Prvi model): [5, 11, 17, 33, 40, 41, 64, 75, 78, 91] 10
Vreme izvršavanja (Prvi model): 592.673773765564 sekundi


C:\Users\Korisnik\AppData\Local\Temp\ipykernel_387984\1791757485.py:2: SyntaxWarning: invalid escape sequence '\c'
  G = load_graph('grafovi\csp\csp100.txt')


KeyboardInterrupt: 